In [25]:
import numpy as np
import h5py
import math
from sklearn.decomposition import PCA
from pca import pca
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm
import peakutils
import os, sys
from BaselineRemoval import BaselineRemoval
from sklearn.cluster import DBSCAN
import scipy
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [26]:
class HiddenPrints:
	def __enter__(self):
		self._original_stdout = sys.stdout
		sys.stdout = open(os.devnull, 'w')

In [27]:
def __exit__(self, exc_type, exc_val, exc_tb):
	sys.stdout.close()
	sys.stdout = self._original_stdout

In [28]:
def remove_outliers(xset, yset, zset, eset, threshold):
	"""
	Uses DBSCAN to find and remove outliers in 3D data
	"""
	data = np.array([xset.T, yset.T, zset.T]).T
	DBSCAN_cluster = DBSCAN(eps=7, min_samples=10).fit(data)
	out_of_cluster_index = np.where(DBSCAN_cluster.labels_==-1)
	del data
	rev = out_of_cluster_index[0][::-1]
	#if len(out_of_cluster_index[0]) > 0:
	for i in rev:
		xset = np.delete(xset, i)
		yset = np.delete(yset, i)
		zset = np.delete(zset, i)
		eset = np.delete(eset, i)
	#if len(xset) <= threshold:
	#	veto = True
	#else:
    #veto = False
	return xset, yset, zset, eset, False

In [29]:
print('Veto on Length @ 70 mm')
print('Veto on Integrated Charge @ 800k')
def track_len(xset, yset, zset):
    """
    Uses PCA to find the length of a track
    """
    veto_on_length = False
 
    # Form data matrix
    data = np.concatenate((xset[:, np.newaxis], 
                           yset[:, np.newaxis], 
                           zset[:, np.newaxis]), 
                           axis=1)

    # Use PCA to find track length
    pca = PCA(n_components=3)
    principalComponents = pca.fit(data)
    principalComponents = pca.transform(data)
    principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])
    
    track_len = 2.35*principalDf.std()[0]
    track_width = 2.35*principalDf.std()[1]
    track_depth = 2.35*principalDf.std()[2]
    #if track_len > 70:
    #    veto_on_length = True
    
    return track_len, veto_on_length, track_width, track_depth

Veto on Length @ 70 mm
Veto on Integrated Charge @ 800k


In [30]:
def main(h5file, threshold):
    """
    This functions does the following: 
    - Converts h5 files into ndarrays. 
    - Removes outliers.
    - Calls PCA to return track length.
    - Sums mesh signal to return energy.
    """
    # Converts h5 files into ndarrays, and output each event dataset as a separte list
    num_events = int(len(list(h5file.keys()))/2) 
    
    len_list = []
    width_list = []
    depth_list = []
    good_events = []
    tot_energy = []
    tracemax_list = []
    tracedev_list = []
    pads_list = []
    
    skipped_events = 0
    veto_events = 0
    
    pbar = tqdm(total=num_events+1)
    for i in range(0, num_events):
        str_event = f"Event_[{i}]"
        
        # Apply pad threshold
        event = np.array(h5file[str_event][:])
        if len(event) <= threshold:
            skipped_events += 1
            pbar.update(n=1)
            continue
            
        # Make copy of datasets
        dset_0_copyx = event['x']
        dset_0_copyy = event['y'] 
        dset_0_copyz = event['z'] - min(event['z'])
        dset_0_copye = event['A']

        
        # Apply veto condition
        R = 36                           # Radius of the pad plane
        r = np.sqrt(dset_0_copyx**2 + dset_0_copyy**2)
        statements = np.greater(r, R)    # Check if any point lies outside of R
      
        if np.any(statements) == True:
            veto_events += 1
            pbar.update(n=1)
            continue
        
        
        # Call remove_outliers to get dataset w/ outliers removed
        dset_0_copyx, dset_0_copyy, dset_0_copyz, dset_0_copye, veto = remove_outliers(dset_0_copyx, dset_0_copyy, dset_0_copyz, dset_0_copye, threshold)
        if veto == True:
            skipped_events += 1
            pbar.update(n=1)
            continue

        
        # Call track_len() to create lists of all track lengths
        length, veto_on_length, width, depth = track_len(dset_0_copyx, dset_0_copyy, dset_0_copyz)
        if veto_on_length == True:
            veto_events += 1
            pbar.update(n=1)
            continue

        
       	str_trace = f"Trace_[{i}]"
        trace = np.array(h5file[str_trace][:])
        max_val = np.argmax(trace)
        low_bound = max_val - 75
        if low_bound < 0:
            low_bound = 5
        upper_bound = max_val + 75
        if upper_bound > 511:
            upper_bound = 506
        trace = trace[low_bound:upper_bound]

        polynomial_degree=2 
        baseObj=BaselineRemoval(trace)
        trace=baseObj.IModPoly(polynomial_degree)

        #if np.sum(trace) > 800000:
        #    veto_events += 1
        #    pbar.update(n=1)
        #    continue

        len_list.append(length)
        width_list.append(width)
        depth_list.append(depth)
        tot_energy.append(np.sum(trace))
        tracemax_list.append(np.max(trace))
        tracedev_list.append(np.std(trace))
        pads_list.append(len(trace[trace > 0]))

        # Track event number of good events
        good_events.append(i)  
        pbar.update(n=1)

    return (tot_energy, skipped_events, veto_events, good_events, len_list, width_list, depth_list, tracemax_list, tracedev_list, pads_list)

In [31]:
# assign directory
directory = "C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/"

output_df = pd.DataFrame(columns=['file','event','length', 'width', 'depth', 'tracesum', 'tracemax', 'tracedev', 'padnum'])

file_names = []
# iterate over files in directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        if f[-3:] == '.h5':
            print(f)
            file_names.append(filename)
            h5f = h5py.File(directory+filename, 'r')
            (tot_energy, skipped_events, veto_events, good_events, len_list, width_list, depth_list, tracemax_list, tracedev_list, pads_list) = main(h5file=h5f, threshold=15)
            for event in range(len(tot_energy)):
                output_df = output_df.append({'file' : filename, 
                                              'event' : event, 
                                              'length' : len_list[event], 
                                              'width' : width_list[event], 
                                              'depth' : depth_list[event], 
                                              'tracesum' : tot_energy[event], 
                                              'tracemax' : tracemax_list[event], 
                                              'tracedev' : tracedev_list[event], 
                                              'padnum' : pads_list[event]}
                                             ,ignore_index = True)


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/273_800p.h5


100%|█████████▉| 676/677 [00:13<00:00, 50.07it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning4protons99-800p.h5


 98%|█████████▊| 50/51 [00:00<00:00, 1469.97it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning5protons0-800p.h5


...

 96%|█████████▌| 25/26 [00:00<00:00, 41.25it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning5protons8-800p.h5


 96%|█████████▌| 25/26 [00:00<00:00, 38.34it/s]


C:/Users/Adam/OneDrive - Tenundra, Inc/Desktop/Programming/GADGET2/simOutput/tuning5protons9-800p.h5


 96%|█████████▌| 25/26 [00:00<00:00, 40.07it/s]


In [32]:
for file in file_names:
    print(file)
    print('length:\t', output_df[output_df['file'] == file]['length'].mean(),'\t', output_df[output_df['file'] == file]['length'].std())
    print('width:\t', output_df[output_df['file'] == file]['width'].mean(),'\t', output_df[output_df['file'] == file]['width'].std())
    print('depth:\t', output_df[output_df['file'] == file]['depth'].mean(),'\t', output_df[output_df['file'] == file]['depth'].std())
    print('tracesum:\t', output_df[output_df['file'] == file]['tracesum'].mean(),'\t', output_df[output_df['file'] == file]['tracesum'].std())
    print('tracemax:\t', output_df[output_df['file'] == file]['tracemax'].mean(),'\t', output_df[output_df['file'] == file]['tracemax'].std())
    print('tracedev:\t', output_df[output_df['file'] == file]['tracedev'].mean(),'\t', output_df[output_df['file'] == file]['tracedev'].std())
    print('padnum:\t', output_df[output_df['file'] == file]['padnum'].mean(),'\t', output_df[output_df['file'] == file]['padnum'].std())
    print()

273_800p.h5
length:	 13.598839169264545 	 0.36378981766204926
width:	 5.490411630687277 	 1.079046393352792
depth:	 1.1956686193850883 	 0.4627512586642447
tracesum:	 155282.26997909456 	 1353.2815542925161
tracemax:	 17261.59640386621 	 3820.7164201418886
tracedev:	 3396.80817140803 	 395.6029936835511
padnum:	 89.44822485207101 	 4.124217163844774

tuning4protons99-800p.h5
length:	 nan 	 nan
width:	 nan 	 nan
depth:	 nan 	 nan
tracesum:	 nan 	 nan
tracemax:	 nan 	 nan
tracedev:	 nan 	 nan
padnum:	 nan 	 nan

tuning5protons0-800p.h5
length:	 14.451866081411987 	 0.712741797259318
width:	 8.432166492070396 	 0.7348868131157585
depth:	 3.6858005032973318 	 0.8998015182220456
tracesum:	 183077.8987832598 	 1758.4729010853905
tracemax:	 17339.632678229806 	 2036.2472388831382
tracedev:	 3680.951514332404 	 249.68365611928175
padnum:	 100.68 	 13.047094184785617

tuning5protons1-800p.h5
length:	 13.968627206319416 	 0.6927059970250923
width:	 8.026652000954394 	 0.6314768846859348
depth:	 

In [33]:
output_df.to_csv('event summary.csv', index=False)

In [34]:
output_df = pd.read_csv('event summary.csv')
file_names = output_df['file'].unique() 

run_df = pd.DataFrame(columns=['file','length', 'width', 'depth', 'tracesum', 'tracemax', 'tracedev', 'padnum'])

reference_run = '273_800p.h5'

for file in file_names:
    run_df = run_df.append({'file' : file, 
                            'length' : output_df[output_df['file'] == file]['length'].mean(), 
                            'width' : output_df[output_df['file'] == file]['width'].mean(), 
                            'depth' : output_df[output_df['file'] == file]['depth'].mean(), 
                            'tracesum' : output_df[output_df['file'] == file]['tracesum'].mean(), 
                            'tracemax' : output_df[output_df['file'] == file]['tracemax'].mean(), 
                            'tracedev' : output_df[output_df['file'] == file]['tracedev'].mean(), 
                            'padnum' : output_df[output_df['file'] == file]['padnum'].mean()}
                           ,ignore_index = True)

run_df['length'] = (run_df['length'] - output_df[output_df['file'] == reference_run]['length'].mean()) / output_df[output_df['file'] == reference_run]['length'].std()
run_df['width'] = (run_df['width'] - output_df[output_df['file'] == reference_run]['width'].mean()) / output_df[output_df['file'] == reference_run]['width'].std()
run_df['depth'] = (run_df['depth'] - output_df[output_df['file'] == reference_run]['depth'].mean()) / output_df[output_df['file'] == reference_run]['depth'].std()
run_df['tracesum'] = (run_df['tracesum'] - output_df[output_df['file'] == reference_run]['tracesum'].mean()) / output_df[output_df['file'] == reference_run]['tracesum'].std()
run_df['tracemax'] = (run_df['tracemax'] - output_df[output_df['file'] == reference_run]['tracemax'].mean()) / output_df[output_df['file'] == reference_run]['tracemax'].std()
run_df['tracedev'] = (run_df['tracedev'] - output_df[output_df['file'] == reference_run]['tracedev'].mean()) / output_df[output_df['file'] == reference_run]['tracedev'].std()
run_df['padnum'] = (run_df['padnum'] - output_df[output_df['file'] == reference_run]['padnum'].mean()) / output_df[output_df['file'] == reference_run]['padnum'].std()

# sum of absolute deviations from reference run
run_df['Deviation'] = abs(run_df['length']) + abs(run_df['width']) + abs(run_df['depth']) + abs(run_df['tracesum']) + abs(run_df['tracemax']) + abs(run_df['tracedev']) + abs(run_df['padnum'])

In [35]:
run_df = run_df.sort_values(by=['Deviation'])
best_run_df = run_df.head(10)

# load in parameters for best runs
params = pd.read_csv('C:\\Users\\Adam\\OneDrive - Tenundra, Inc\\Desktop\\Programming\\GADGET2\\simInput\\parameters.csv')

best_params = params[params['Sim'].isin(best_run_df['file'].str.replace('.h5',''))]

In [36]:
parameters = ['Threshold', 'EIonize', 'Fano', 'CoefL', 'CoefT', 'Gain', 'GETGain', 'PeakingTime']
print('Current Best Fit Parameters')
for p in parameters:
    print(p, '\t', best_params[p].mean(),'\t', best_params[p].std()/best_params[p].mean(), '\n', params[p].mean(),'\t', params[p].std()/params[p].mean(), '\n')

Current Best Fit Parameters
Threshold 	 24.0 	 0.0625 
 23.975 	 0.0516268648751543 

EIonize 	 33.72222222222223 	 0.03865781469593311 
 34.345 	 0.05092669347074938 

Fano 	 0.22333333333333336 	 0.044776119402985086 
 0.22950000000000004 	 0.04576232005875304 

CoefL 	 0.00012355555555555554 	 0.040490101672132114 
 0.00012302 	 0.04934084671379143 

CoefT 	 0.002164444444444444 	 0.040085467407044315 
 0.0021469500000000003 	 0.0484616865068955 

Gain 	 7330.98888888889 	 0.052060813626819746 
 7366.1945 	 0.050594871584115304 

GETGain 	 141.14444444444445 	 0.040895377829311956 
 138.672 	 0.05432822983027321 

PeakingTime 	 551.6666666666666 	 0.041966780312230424 
 560.225 	 0.04842457949507863 



In [37]:
print('Full Run Deviation:', run_df['Deviation'].mean(), '\t', run_df['Deviation'].std()/run_df['Deviation'].mean())
print('Best Run Deviation:', best_run_df['Deviation'].mean(), '\t', best_run_df['Deviation'].std()/best_run_df['Deviation'].mean())

Full Run Deviation: 25.867678178798194 	 0.384349720596579
Best Run Deviation: 14.533072811732083 	 0.37061727029403896


In [38]:
best_run_df.head()

,file,length,width,depth,tracesum,tracemax,tracedev,padnum,Deviation
0,273_800p.h5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
39,tuning5protons43-800p.h5,1.447570,2.361157,5.143063,0.135437,-0.600973,-0.615563,2.820360,13.124122
14,tuning5protons20-800p.h5,2.072719,2.531050,5.606639,-0.354734,-0.687886,-0.734123,2.403311,14.390463
26,tuning5protons31-800p.h5,2.320930,2.664106,5.890829,0.662865,-0.660335,-0.678384,1.928069,14.805518
22,tuning5protons28-800p.h5,2.833775,2.871227,5.780398,0.432582,-0.505308,-0.506415,2.655480,15.585184


In [39]:
run_df.tail()

,file,length,width,depth,tracesum,tracemax,tracedev,padnum,Deviation
2,tuning5protons1-800p.h5,1.016488,2.350446,5.319864,-24.852981,-1.412623,-2.311811,2.393612,39.657825
28,tuning5protons33-800p.h5,1.365719,2.372994,5.839436,-25.687925,-1.503842,-2.437309,1.384936,40.592160
32,tuning5protons37-800p.h5,3.910108,3.144345,6.433048,26.757670,0.240395,1.148433,2.597287,44.231286
3,tuning5protons10-800p.h5,3.368771,3.120827,6.380842,31.662765,0.284862,1.359644,2.413009,48.590722
9,tuning5protons16-800p.h5,2.763728,2.736971,6.462356,34.608235,0.572614,1.761440,1.976563,50.881906


In [40]:
params[params['Sim'] == 'tuning4protons74-800p']

,Sim,Status,N,P0,E0,P1,E1,Xb,Yb,Zb1,Zb2,Threshold,EIonize,Fano,CoefL,CoefT,Gain,GETGain,PeakingTime
